In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Suicide_Detection.csv to Suicide_Detection.csv


In [ ]:
import pandas as pd

# Load the preprocessed dataset
dataset = pd.read_csv("Suicide_Detection.csv")  # Use the correct filename if needed
print(dataset.head())


                                                text        class
0  Im really scaredI always had it in my mind if ...      suicide
1  Ok ok ok ok I have a very important question H...  non-suicide
2  Wanting to stop pretending. I'm done.I have no...      suicide
3  I am tired of being me.I am 22 and have had on...      suicide
4  Am I being overdramatic or should I end it all...      suicide


In [ ]:
pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Preprocessing
dataset['label'] = dataset['class'].map({'suicide': 1, 'non-suicide': 0})  # Encode labels
train_texts, test_texts, train_labels, test_labels = train_test_split(
    dataset['text'].tolist(), dataset['label'].tolist(), test_size=0.2, random_state=42
)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

# Convert to Hugging Face Dataset
train_data = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_data = Dataset.from_dict({'text': test_texts, 'label': test_labels})

train_data = train_data.map(tokenize, batched=True)
test_data = test_data.map(tokenize, batched=True)

train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Load Pretrained BERT Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    eval_strategy="epoch",  # Updated
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"  # Disables WandB logging
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data
)


# Train the Model
trainer.train()

# Evaluate the Model
predictions = trainer.predict(test_data)
y_true = test_data['label']
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1)

print(classification_report(y_true, y_pred))

# Save the Model
model.save_pretrained('./bert-suicide-detection')
tokenizer.save_pretrained('./bert-suicide-detection')




Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.159045
2,0.271100,0.129351
3,0.271100,0.141733


              precision    recall  f1-score   support

           0       0.97      0.96      0.96       513
           1       0.95      0.97      0.96       487

    accuracy                           0.96      1000
   macro avg       0.96      0.96      0.96      1000
weighted avg       0.96      0.96      0.96      1000



('./bert-suicide-detection/tokenizer_config.json',
 './bert-suicide-detection/special_tokens_map.json',
 './bert-suicide-detection/vocab.txt',
 './bert-suicide-detection/added_tokens.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model.save_pretrained('/content/drive/MyDrive/bert-suicide-detection')
tokenizer.save_pretrained('/content/drive/MyDrive/bert-suicide-detection')


('/content/drive/MyDrive/bert-suicide-detection/tokenizer_config.json',
 '/content/drive/MyDrive/bert-suicide-detection/special_tokens_map.json',
 '/content/drive/MyDrive/bert-suicide-detection/vocab.txt',
 '/content/drive/MyDrive/bert-suicide-detection/added_tokens.json')

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained('./bert-suicide-detection')
tokenizer = BertTokenizer.from_pretrained('./bert-suicide-detection')

# Example inference
text = "i hate my self so much that i don't want to live anymore"
tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
outputs = model(**tokens)
prediction = torch.argmax(outputs.logits, dim=1).item()
print("Prediction:", "suicide" if prediction == 1 else "non-suicide")


In [ ]:
import os
print(os.getcwd())  # This will print the current directory in Colab


/content


In [ ]:
!ls /content


 bert-suicide-detection			      logs	    Suicide_Detection.csv
'C:\Users\vishw\OneDrive\Desktop\tweet bot'   results
'C:\Users\vishw\working'		      sample_data
